In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
import multiprocessing
from torch import autograd
from fastai.conv_learner import *
from fasterai.images import *
from fasterai.dataset import *
from fasterai.visualize import *
from fasterai.callbacks import *
from fasterai.loss import *
from fasterai.modules import *
from fasterai.wgan import *
from fasterai.generators import *
from fastai.torch_imports import *
from pathlib import Path
from itertools import repeat
import tensorboardX
torch.cuda.set_device(2)
plt.style.use('dark_background')
torch.backends.cudnn.benchmark=True


In [3]:
DATA_PATH = Path('data/imagenet/ILSVRC/Data/CLS-LOC')
TRAIN_SOURCE_PATH = DATA_PATH/'train'
proj_id = 'bw2color'
TENSORBOARD_PATH = Path('data/tensorboard/WganUnet_RC28')


dpath = DATA_PATH/(proj_id + '_654_D.h5')
gpath = DATA_PATH/(proj_id + '_654_G.h5')
keep_pct=1.0
bs=8
sz=256

c_lr=3e-4
c_lrs = np.array([c_lr,c_lr,c_lr,c_lr])

g_lr=1e-4
g_lrs = np.array([g_lr/10000,g_lr/100,g_lr])

torch.backends.cudnn.benchmark=True

## Training

In [4]:
netG = Unet34(nf_factor=1).cuda()
#netGVis = ModelVisualizationHook(TENSORBOARD_PATH, netG, 'netG')
#load_model(netG, gpath)

netD = DCCritic(ni=3, nf=64, scale=32).cuda()
#netDVis = ModelVisualizationHook(TENSORBOARD_PATH, netD, 'netD')
#load_model(netD, dpath)

In [5]:
md = get_matched_image_model_data(image_size=sz, batch_size=bs, root_data_path=DATA_PATH, train_root_path=TRAIN_SOURCE_PATH, proj_id=proj_id, keep_pct=keep_pct)

In [6]:
trainer = WGANTrainer(netD=netD, netG=netG, md=md, bs=bs, sz=sz, dpath=dpath, gpath=gpath, genloss_fns=[FeatureLoss(multiplier=1e5)], citers=5)
trainerVis = WganVisualizationHook(TENSORBOARD_PATH, trainer, 'trainer')

In [ ]:
#netG.freeze_to(1)
trainer.train(lrs_critic=c_lrs, lrs_gen=g_lrs, clr_critic=(1,8), clr_gen=(1,8), cycle_len=1, epochs=1)

  0%|          | 0/1 [00:00<?, ?it/s]

/media/jason/Projects/Deep Learning/Deep Learning Projects/Jupyter Experiments/1_BlackWhiteToColor/fasterai/generators.py:131: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if h<target_h or w<target_w:


Failed to generate graph for model: %426 : Long() = onnx::Constant[value={2}]()
 has empty scope name. Note that there's an outstanding issue with scopes being addressed here:  https://github.com/pytorch/pytorch/pull/12400
  0%|          | 60/158545 [00:31<16:38:25,  2.65it/s]
WDist -5.514005897566676e-05; RScore -0.0003011325898114592; FScore -0.00035627264878712595; GAddlLoss 6569.744140625; GCount: 10; GPenalty: 9.999999046325684; GCost: 0.0002945551532320678; ConPenalty: 0.0004161386168561876
  0%|          | 120/158545 [00:54<16:32:43,  2.66it/s]
WDist -2.9708564397878945e-05; RScore -0.00019268185133114457; FScore -0.00022239041572902352; GAddlLoss 6576.978515625; GCount: 20; GPenalty: 9.999998092651367; GCost: 0.00025716444361023605; ConPenalty: 0.000470996048534289
  0%|          | 180/158545 [01:25<16:51:35,  2.61it/s]
WDist -9.334676724392921e-05; RScore -9.813961514737457e-05; FScore -0.00019148638239130378; GAddlLoss 8059.67529296875; GCount: 30; GPenalty: 9.99999713897705;